In [1]:
import os
import google.generativeai as genai  # type: ignore


c:\Users\mitaksh\New folder\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
genai.configure(api_key="#YOUR API KEY HERE")

generation_config = {
    'temperature': 0.7,
    'top_p': 0.95,
    "top_k": 40,
    "max_output_tokens": 200,
    "response_mime_type": "text/plain"
}

my_bot = genai.GenerativeModel(
    model_name='gemini-1.5-flash',  # Use correct available model
    generation_config=generation_config,
    system_instruction = """
You are a bug classification assistant. You will be given **buggy Python code with line numbers**. Your task is to:

1. Carefully analyze the given code.
2. Identify the type of bug in the code using only the following list of 14 bug types.
3. Assume there is always **exactly one bug** in the input code.
4. Return your output strictly in the following JSON format:

{
  "bug_type": "<one of the 14 types below>"
}

Valid bug types:
- Incorrect assignment operator  
- Incorrect variable  
- Incorrect comparison operator  
- Missing condition  
- Missing/added +1  
- Variable swap  
- Incorrect array slice  
- Variable prepend  
- Incorrect data structure constant  
- Incorrect method called  
- Incorrect field dereference  
- Missing arithmetic expression  
- Missing function call  
- Missing line

🧠 Use logic and context to identify how the bug affects the code.
❌ Do not explain the answer or modify the code.
✅ Return only the bug type in JSON format.

If the code looks fine, still pick the **most likely bug** from the list.

Example:

Input:
1. def add_one(x):  
2.     return x + 2  

Output:
{
  "bug_type": "Missing/added +1"
}
"""
)

In [ ]:
import re
import json

# Your chatbot setup
history = []
print("bot : HELLO, HOW CAN I HELP YOU")
chat_session = my_bot.start_chat(history=history)

# Base system prompt with few-shot examples + instruction to always return bug type only
base_system_prompt = """
You are a bug detection agent. You will be given buggy Python code with line numbers. Your task is to:

1. Identify the type of bug in the code.

Choose one of these categories only:

- Incorrect assignment operator
- Incorrect variable
- Incorrect comparison operator
- Missing condition
- Missing/added +1
- Variable swap
- Incorrect array slice
- Variable prepend
- Incorrect data structure constant
- Incorrect method called
- Incorrect field dereference
- Missing arithmetic expression
- Missing function call
- Missing line

Return your result in this JSON format only:
{
  "bug_type": "..."
}

Important:
- Always assume there is exactly one bug.
- Do NOT include line numbers.
- Do NOT add any extra text or explanation.

Examples:

Code:
1. def is_even(n):
2.     return n % 2 = 0

Output:
{
  "bug_type": "Incorrect assignment operator"
}

Code:
1. def gcd(a, b):
2.     if b == 0:
3.         return a
4.     else:
5.         return gcd(a % b, b)

Output:
{
  "bug_type": "Incorrect variable"
}

Now analyze this code:
"""

# Utility: Add line numbers to code
def add_line_numbers(code: str) -> str:
    return '\n'.join([f"{i+1}. {line}" for i, line in enumerate(code.splitlines())])

# Utility: Extract JSON bug type from LLM output safely
def extract_bug_info(text: str):
    try:
        match = re.search(r'\{.*?\}', text, re.DOTALL)
        if match:
            json_text = match.group()
            bug_info = json.loads(json_text)
            return bug_info
    except json.JSONDecodeError:
        return {"bug_type": "ParseError"}
    return {"bug_type": "None"}

# Main loop
while True:
    print("\nOptions:\n1. Type code\n2. Load from file\n(Type 'exit' to quit)")
    choice = input("Choose option (1 or 2): ").strip()

    if choice.lower() in ["exit", "quit", "bye"]:
        print("bot : Goodbye! Have a great day!")
        break

    elif choice == "1":
        user_input = input("Paste your buggy code below:\n")
        code = user_input

    elif choice == "2":
        file_path = input("Enter path to your Python file: ").strip()
        try:
            with open(file_path, 'r') as f:
                code = f.read()
        except FileNotFoundError:
            print("bot : File not found. Please try again.")
            continue

    else:
        print("bot : Invalid option. Please choose 1 or 2.")
        continue

    # Add line numbers to code
    numbered_code = add_line_numbers(code)
    full_prompt = base_system_prompt + numbered_code

    print("\nSending this code to the LLM:\n")
    print(numbered_code)

    # Ask the LLM to classify the bug using the full prompt with examples
    response = chat_session.send_message(full_prompt)
    model_response = response.text

    # Save to history
    history.append({"user": code, "bot": model_response})

    # Extract bug info (only bug_type)
    bug_result = extract_bug_info(model_response)

    print("\nbot : Detected bug type:")
    bug_type = bug_result.get("bug_type")
    print(bug_result.get("bug_type"))


In [4]:
my_bot = genai.GenerativeModel(
    model_name='gemini-1.5-flash',
    generation_config=generation_config,
    system_instruction= """
You are a bug fixing assistant. You will be given Python code with exactly one bug in it, along with the type of bug.

Your task is to:
1. Carefully understand the code and the provided bug type.
2. Fix the bug in the code.
3. Return ONLY the corrected code — do not explain the changes or include any extra text.

You must fix one of the following 14 bug types:
- Incorrect assignment operator  
- Incorrect variable  
- Incorrect comparison operator  
- Missing condition  
- Missing/added +1  
- Variable swap  
- Incorrect array slice  
- Variable prepend  
- Incorrect data structure constant  
- Incorrect method called  
- Incorrect field dereference  
- Missing arithmetic expression  
- Missing function call  
- Missing line

🔧 The input will always contain:
- A `bug_type` field (with one of the 14 types above)
- Python code with the bug

🧠 Think carefully and return the fixed code in proper Python syntax.

📤 Output format:
Return ONLY the fixed code. No explanations, no JSON, no markdown — just valid Python code.

Example Input:
bug_type: Missing/added +1  
code:
1. def add_one(x):  
2.     return x + 2

Example Output:
def add_one(x):
    return x + 1
"""

)

In [ ]:
code_input = numbered_code
bug = bug_type
print(bug)
import re



# --------- Bug Fixer Prompt Template ---------
FIXER_PROMPT = """
You are a bug-fixing assistant. You will be given:

- Buggy Python code
- The type of bug it contains (from a predefined list)

Your job is to:
- Analyze the buggy code
- Fix the code based ONLY on the given bug type
- Return only the fixed code (without explanation, comments, or extra formatting)

Here are the 14 bug types you can encounter and how to fix them:

1. Incorrect assignment operator — Fix incorrect usage of '=' vs '==' or similar assignment logic.
2. Incorrect variable — Replace the wrong variable used with the correct one based on context.
3. Incorrect comparison operator — Replace operators like '<', '>', '==', etc., with the correct one.
4. Missing condition — Add an if/else condition that is logically required.
5. Missing/added +1 — Fix off-by-one errors, such as loop indices or ranges.
6. Variable swap — Swap two variables that were incorrectly ordered.
7. Incorrect array slice — Correct the slicing bounds or syntax of lists or arrays.
8. Variable prepend — Add a variable (e.g., 'self.' or function argument) that was missing before usage.
9. Incorrect data structure constant — Fix incorrect usage of default values (e.g., using {{}} vs [] or 0).
10. Incorrect method called — Replace the incorrect function or method with the correct one.
11. Incorrect field dereference — Fix incorrect access of an object attribute or field.
12. Missing arithmetic expression — Add a necessary operation like '+', '-', '*', etc.
13. Missing function call — Call a function that was referenced but not invoked.
14. Missing line — Insert the missing line needed for the code to work properly.

Example 1:
Bug Type: Incorrect variable  
Code:
```python
def square(n):
    return x * x
```

Fix:
```python
def square(n):
    return n * n
```

Example 2:
Bug Type: Missing function call  
Code:
```python
def length(lst):
    return lst
```

Fix:
```python
def length(lst):
    return len(lst)
```

Now fix this:
Bug Type: {bug_type}  
Code:
```python
{buggy_code}
```

Return only the fixed code.
"""

# --------- Bug Fixing Function ---------
def fix_buggy_code(my_bot, buggy_code: str, bug_type: str) -> str:
    prompt = FIXER_PROMPT.format(
        bug_type=bug_type,
        buggy_code=buggy_code.strip()
    )
    chat = my_bot.start_chat()
    response = chat.send_message(prompt)
    return response.text.strip()

# --------- Example CLI Interface ---------
if __name__ == "__main__":
    print("=== Bug Fixer Agent ===")
    while True:
        buggy_code = code_input
        bug_type = bug

        print("\nFixing bug...")
        try:
            fixed_code = fix_buggy_code(my_bot, buggy_code, bug_type)
            print("\n=== Fixed Code ===")
            print(fixed_code)
        except Exception as e:
            print("Error during bug fixing:", str(e))
